In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import urllib.request
from pulp import *
from IPython.display import Image, display_png
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib inline

In [1]:
t = pd.read_csv('../csv/stochastic_matrix.csv', index_col=0).fillna(0)

NameError: name 'pd' is not defined

In [3]:
v = t.T[[
    'R01786','R02740','R04779','R01070','R01061','R01512','R01518','R00658',
    'R00200','R01196','R00351','R00341','R01325','R01900','R01899','R00268',
    'R01197','R00405','R02164','R01082','R00342','R01015','R00703','R02035',
    'R01528','R01056','R01049','R01529','R01641', 'R01830',
]].T

for i in v.columns:
    if sum(v[i] == 0) == v.shape[0]:
        v = v.drop(i, axis=1)
        
v['C00007'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00033'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00092'] = pd.Series(np.zeros(v.shape[0]), index=v.index)
v['C00279'] = pd.Series(np.zeros(v.shape[0]), index=v.index)

In [4]:
df = v.T
df['Glucose'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glucose']['C00267'] = 1

df['Oxygen'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxygen']['C00007'] = 1

df['CO2'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['CO2']['C00011'] = -1

df['Lactate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Lactate']['C00186'] = -1

df['Succinate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Succinate']['C00042'] = -1

df['Malate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Malate']['C00149'] = -1

df['Fumarate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Fumarate']['C00122'] = -1

df['Acetate'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Acetate']['C00033'] = -1

df['Glu_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Glu_Exchange']['C00267'] = -1
df['Glu_Exchange']['C00092'] = 1

df['acetyl_CoA_hydrolase'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['acetyl_CoA_hydrolase']['C00024'] = -1
df['acetyl_CoA_hydrolase']['C00001'] = -1
df['acetyl_CoA_hydrolase']['C00010'] = 1
df['acetyl_CoA_hydrolase']['C00033'] = 1

df['NADP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['NADP_Exchange']['C00005'] = -1
df['NADP_Exchange']['C00003'] = -1
df['NADP_Exchange']['C00006'] = 1
df['NADP_Exchange']['C00004'] = 1

df['AMP_Exchange'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['AMP_Exchange']['C00002'] = -1
df['AMP_Exchange']['C00020'] = -1
df['AMP_Exchange']['C00008'] = 2

# df['R00835'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
# df['R00835']['C00092'] = -1
# df['R00835']['C00006'] = -1
# df['R00835']['C01236'] = 1
# df['R00835']['C00005'] = 1
# df['R00835']['C00080'] = 1

df['R08575'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['R08575']['C05382'] = -1
df['R08575']['C00118'] = -1
df['R08575']['C00279'] = 1
df['R08575']['C00085'] = 1

df['R02164']['C00004'] = -1
df['R02164']['C00003'] = 1
df['R02164']['C15602'] = 0
df['R02164']['C15603'] = 0

# Oxophos
df['Oxophos'] =  pd.Series(np.zeros(df.shape[0]), index=df.index)
df['Oxophos']['C00004'] = -2
df['Oxophos']['C00007'] = -1
df['Oxophos']['C00008'] = -5
df['Oxophos']['C00002'] = 5
df['Oxophos']['C00003'] = 2
df['Oxophos']['C00011'] = 1

In [5]:
BIOMASS = pd.read_csv('../csv/amino_acid.csv',index_col=0)
BIOMASS = BIOMASS[BIOMASS.index != 'NH3']
df_Biomass =  pd.Series(np.zeros(df.shape[0]), index=df.index)

for i in BIOMASS.index:
    df_Biomass[BIOMASS.T[i]['C_num']] = BIOMASS.T[i]['flux']
    
df_Biomass['C00668'] = -0.27519
df_Biomass['C00119'] = (-0.039036) + (-0.036117) + (-0.053446) + (-0.039036)

In [6]:
df_new = pd.concat([df_Biomass.T, df], axis=1)
df_new = df_new.rename(columns={0: 'BIOMASS'})
df_new = df_new.drop([
    'C00001','C00009','C00080','C00138','C00139','C15602','C15603', 
#     'C00231'
])

In [7]:
x = []
for i in df_new/columns:
    if (
            (i=='Oxophos') or (i=='R01899') or (i=='R00268') or (i=='R10343') or (i=='R02164') or (i=='R00835') or (i=='R02035') or
            (i=='CO2') or (i == 'Lactate') or (i == 'Acetate') or (i == 'Succinate') or (i == 'Malate') or (i=='ATP_Exchange') or (i=='AMP_Exchange') or
            (i=='Glutamate') or (i=='CO2') or  (i=='Lactate') or (i=='Succinate') or 
            (i=='Malate') or (i=='Acetate') or (i=='Glutamate') or (i=='Fumarate')
    ) :
        x.append(LpVariable(i, lowBound=0,upBound=10000, cat='Continuous'))

    elif (i=='Glucose'):
        x.append(LpVariable(i, lowBound=0, upBound=100000, cat='Continuous'))

    elif (i=='Glutamate'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))
        
    elif (i=='Oxygen'):
        x.append(LpVariable(i, lowBound=0, upBound=100, cat='Continuous'))

    elif (i=='R00200') or (i=='R01196') or (i=='R01197') or (i=='R00405'):
        x.append(LpVariable(i, lowBound=-10000, upBound=0, cat='Continuous'))

    else:
        x.append(LpVariable(i, lowBound=-10000, upBound=10000, cat='Continuous'))

In [8]:
m = LpProblem(sense=LpMaximize)
m += x[0]

for i in np.dot(df, x):
    m += i == 0

In [9]:
status = m.solve()
print(LpStatus[status])

Optimal


In [10]:
y = []
for i in x:
    y.append(i.value())
    
pd.DataFrame(y, index=[str(i) for i in x]).sort_index()

,0
AMP_Exchange,0.000000
Acetate,0.000000
Biomass,39.296259
CO2,0.000000
Fumarate,0.000000
Glu_Exchange,0.000000
Glucose,217.196580
Lactate,310.079350
Malate,100.000000
NADP_Exchange,123.137220
